# Naive Bayes Classifier

## 1 - Packages ##


In [1]:
import pandas as pd
import numpy as np
import re

## 2 - Overview of the Problem set ##

**Problem Statement**: given a dataset containing:
    - a training set of m_train examples
    - a test set of m_test examples
    - each example is a message that belongs to a particular class: ham or spam.


In [2]:
# Loading the data 

def load_data():
    df = pd.read_csv('http://brodaua.github.io/university-labs/spam.csv', encoding='latin-1')
    #df = pd.read_csv('spam.csv', encoding='latin-1')
    df_for_tests = df.head()
    
    idx = np.arange(df.shape[0])
    np.random.shuffle(idx)

    train_set_size = int(df.shape[0] * 0.8)

    train_set = df.loc[idx[:train_set_size]]
    test_set = df.loc[idx[train_set_size:]]
    
    return train_set, test_set, df_for_tests

In [3]:
train_set, test_set, df_for_tests = load_data()

## 3 - Naive Bayes Classifier

This algorithm is based on Bayes' theorem: 
    $$ \begin{equation} \\
    P(A_{j}\hspace{0.1cm}\rvert\hspace{0.1cm}x_{1},\dots,x_{n}) = \frac{P(x_{1},\dots,x_{n}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j})P(A_{j})}{P(x_{1},\dots,x_{n})} \\
  =>
    P(A_{j})P(x_{1},\dots,x_{n}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j}) = P(A_{j}, x_{1},\dots,x_{n})
   = P(x_{1}\hspace{0.1cm}\rvert\hspace{0.1cm}x_{2},\dots,x_{n}, A_{j})P(x_{2}\hspace{0.1cm}\rvert\hspace{0.1cm}x_{3}, \dots ,x_{n}, A_{j})\dots P(x_{n-1}\hspace{0.1cm}\rvert\hspace{0.1cm}x_{n}, A_{j})
   \approx P(A_{j}) \prod_{i=1}^{n} P(x_{i}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j})
   \end{equation}$$
   
We can calculate the probability, if we know the prior probability:

$$ \begin{equation}
    y^{*} = \operatorname*{arg\,max}_{j} \big(P(A_{j}) \prod_{i=1}^{n} P(x_{i}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j})\big)
    \hspace{1cm} 
    y^{*} = \operatorname*{arg\,max}_{j} \big( \ln(P(A_{j})) + \sum_{i=1}^{n} \ln(P(x_{i}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j})) \big)
   \end{equation}$$
   
   
#### Laplace smoothing

$$ \begin{equation}
    \theta_i = \frac{x_{i} + \alpha}{N + \alpha k}
   \end{equation}$$

where the pseudocount 
$\begin{equation}
    \alpha > 0
 \end{equation}$ is the smoothing parameter (
$\begin{equation}
    \alpha = 0
 \end{equation}$ corresponds to no smoothing).

### 3.1 - Preprocessing the data 

Our data consists of different messages. Messages contain some excess symbols, which don't affect the content of the text, but add noise to the data.
For example: "Does not \\operate 66.7 after  & lt;# & gt;  or what". 

Let's clean our data and leave only letters a-z, A-Z, numbers 0-9 and cast all letters to lowercase, replace double to n spaces with just one space, remove trailing spaces.

In [4]:
# Clean the data

def clean_data(message):
    
    return re.sub('[ ]+',' ', ''.join(re.findall('[a-z0-9 ]*', message, flags=re.I | re.ASCII))).lower()

In [5]:
sentence = 'Does not  \\operate 66.7 af\'ter & lt;# & gt; or what'
print('cleaned: ',clean_data(sentence))

cleaned:  does not operate 667 after lt gt or what


In [6]:
# Preparation data for model

def prep_for_model(train_set, test_set):
    train_set_x = np.array([clean_data(s[0]).split(' ') for s in train_set.iloc[:,1:].values])
    train_set_y = train_set.iloc[:,0].values
    test_set_x = np.array([clean_data(s[0]).split(' ') for s in test_set.iloc[:,1:].values])
    test_set_y = test_set.iloc[:,0].values
      
    return train_set_x, train_set_y, test_set_x, test_set_y

train_set_x, train_set_y, test_set_x, test_set_y = prep_for_model(train_set, test_set)

In [7]:
a1, a2, b1, b2 = prep_for_model(df_for_tests.head(3), df_for_tests.tail(2))
print(a2[0], a1[0])
print(b2[0], b1[0])

ham ['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'there', 'got', 'amore', 'wat']
ham ['u', 'dun', 'say', 'so', 'early', 'hor', 'u', 'c', 'already', 'then', 'say']


In [8]:
# Check words in categories

def categories_words(x_train, y_train):
    
    all_words_list = []
    ham_words_list = []
    spam_words_list = []
    
    x_train=np.array(x_train)
    y_train=np.array(y_train)
    
    all_words_list = np.concatenate(x_train)
    spam_words_list = np.concatenate(x_train[y_train == 'spam'])
    ham_words_list = np.concatenate(x_train[y_train == 'ham'])
        
    return all_words_list, ham_words_list, spam_words_list

all_words_list_a1, ham_words_list_a1, spam_words_list_a1 = categories_words(a1, a2)

In [9]:
print('first five "ham" words of a1: ', ham_words_list_a1[:5])

first five "ham" words of a1:  ['go' 'until' 'jurong' 'point' 'crazy']


### 3.2 Model

In [42]:
class Naive_Bayes(object):

    def __init__(self, alpha):
        self.alpha = alpha
        
        self.train_set_x = None
        self.train_set_y = None
        
        self.all_words_list = []
        self.ham_words_list = []
        self.spam_words_list = []
    
    def estimator(self, X):
        values, frequency = np.unique(X, return_counts=True)
        
        N = X.shape[0]
        k = np.unique(self.all_words_list).shape[0]
        
        res = pd.DataFrame(data = frequency, index=values, columns=['frequency'])
        return res.assign(probability = lambda x: (x+self.alpha)/(N+self.alpha*k))       
      
    def fit(self, train_set_x, train_set_y):
        
        # Generate all_words_list, ham_words_list, spam_words_list using function 'categories_words'; 
        # Calculate probability of each word in both categories
    
        self.all_words_list, self.ham_words_list, self.spam_words_list = categories_words(train_set_x, train_set_y)
        
        self.ham_words_probs = self.estimator(self.ham_words_list)
        self.spam_words_probs = self.estimator(self.spam_words_list)
        
        labels, counts = np.unique(train_set_y, return_counts=True)
        self.class_probs = {l:c/sum(counts) for l,c in zip(labels, counts)}
        
    def predict(self, test_set_x):
                
        def infer(X):
            X = np.array(X)

            spam = [self.class_probs['spam']]
            ham = [self.class_probs['ham']]
           
            #unique word approach
            ham.extend(self.ham_words_probs[np.isin(self.ham_words_probs.index, X)]['probability'].values)
            spam.extend(self.spam_words_probs[np.isin(self.spam_words_probs.index, X)]['probability'].values)
            
                
            # + repeated word approach
            
            X_unique, counts = np.unique(X, return_counts=True)
            
            h = self.ham_words_probs[np.isin(self.ham_words_probs.index, X)]['probability']
            s = self.spam_words_probs[np.isin(self.spam_words_probs.index, X)]['probability']
            
            ham.extend((h*counts[np.isin(X_unique, h.index)]).values)
            spam.extend((s*counts[np.isin(X_unique, s.index)]).values)
            
            return 'spam' if np.sum(np.log(spam)) > np.sum(np.log(ham)) else 'ham'
            
        return np.vectorize(infer)(test_set_x)    

## 4 - Training

In [11]:
a = 1
model = Naive_Bayes(alpha=a)
model.fit(train_set_x, train_set_y)

## 5 - Making predictions

In [47]:
y_predictions = model.predict(test_set_x)

In [46]:
actual = list(test_set_y)
accuracy = (y_predictions == test_set_y).mean()
print(accuracy)

0.8717488789237668
